# 部分採用，擷取特徵

In [ ]:
# -*- coding: utf-8 -*-
#-*- coding: cp950 -*-

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os

from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)
base_dir = '/content/gdrive/MyDrive'
path = Path(base_dir +'/Keras_tutorial')  #imgs
# path.mkdir(parents=True,exist_ok=True)
os.chdir(path)

Mounted at /content/gdrive


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

# 預先訓練好的模型 -- VGG16, 不含後三層(辨識層)
model = VGG16(weights='imagenet', include_top=False)

# 任意一張圖片，例如大象
img_path = './images/elephant.jpg'
# 載入圖檔，並縮放寬高為 (224, 224) 
img = image.load_img(img_path, target_size=(224, 224))
# 加一維，變成 (1, 224, 224, 3)，最後一維是色彩
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
print(features)

[[[[ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.        42.547775  ...  0.         0.
     0.       ]
   [ 1.0753291  0.        23.49561   ...  0.         0.
     0.       ]
   ...
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]]

  [[ 0.         0.        36.33886   ...  0.         0.
     3.4028866]
   [ 0.         0.        80.2363    ...  7.871879   0.
     0.       ]
   [ 0.         0.        48.7513    ...  0.         0.
     0.       ]
   ...
   [ 0.         0.         0.        ...  4.5813546  0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]]

  [[ 0.         0.         9.853476  ...  0.         0.
     2.4919636]
   [ 0.         0.        25.24964   ... 42.27

In [ ]:
print(features.shape)

(1, 7, 7, 512)


In [ ]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [ ]:
model2 = VGG16(weights='imagenet', include_top=True)
model2.summary()

553467904/553467096 [==============================] - 11s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________________

In [ ]:
print(features)

[[[[ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.        42.547775  ...  0.         0.
     0.       ]
   [ 1.0753291  0.        23.49561   ...  0.         0.
     0.       ]
   ...
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]]

  [[ 0.         0.        36.33886   ...  0.         0.
     3.4028866]
   [ 0.         0.        80.2363    ...  7.871879   0.
     0.       ]
   [ 0.         0.        48.7513    ...  0.         0.
     0.       ]
   ...
   [ 0.         0.         0.        ...  4.5813546  0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]]

  [[ 0.         0.         9.853476  ...  0.         0.
     2.4919636]
   [ 0.         0.        25.24964   ... 42.27

In [ ]:
print(features.shape)

(1, 7, 7, 512)


# 使用 cosine_similarity 比較特徵向量

In [ ]:
from os import listdir
from os.path import isfile, join

# 讀取 images 目錄下所有圖檔
img_path = './images/'
image_files = np.array([f for f in listdir(img_path) if isfile(join(img_path, f)) and f[-3:] == 'jpg'])
image_files

array(['Tiger3.jpg', 'with-mask.jpg', 'without-mask.jpg', 'input.jpg',
       'style.jpg', 'bird01.jpg', 'Tiger2.jpg', 'Tiger.jpg',
       'elephant.jpg', '太陽花.jpg', 'elephant2.jpg'], dtype='<U16')

In [ ]:
# 每個圖檔經過預測，取得特徵向量
import numpy as np

X = np.array([])
# 合併每個圖檔的像素
for f in image_files:
    image_file = join(img_path, f)
    # 載入圖檔，並縮放寬高為 (224, 224) 
    img = image.load_img(image_file, target_size=(224, 224))
    # 加一維，變成 (1, 224, 224, 3)，最後一維是色彩
    img2 = image.img_to_array(img)
    img2 = np.expand_dims(img2, axis=0)
    if len(X.shape) == 1:
        X = img2
    else:
        X = np.concatenate((X, img2), axis=0)

X = preprocess_input(X)

# 預測
features = model.predict(X)

features.shape, X.shape

((11, 7, 7, 512), (11, 224, 224, 3))

In [ ]:
# 使用 cosine_similarity 比較特徵向量
from sklearn.metrics.pairwise import cosine_similarity

features2 = features.reshape((features.shape[0], -1))

# 比較對象：Tiger3.jpg
no=-2
print(image_files[no])
similar_list = cosine_similarity(features2[no:no+1], features2[:no], dense_output=False)
print(np.sort(similar_list[0])[::-1])
image_files[:no][np.argsort(similar_list[0])[::-1]]

太陽花.jpg
[0.10594285 0.08696808 0.06012522 0.05226007 0.03927399 0.03698231
 0.03596732 0.03461865 0.02446406]


array(['bird01.jpg', 'style.jpg', 'Tiger2.jpg', 'with-mask.jpg',
       'Tiger3.jpg', 'elephant.jpg', 'without-mask.jpg', 'Tiger.jpg',
       'input.jpg'], dtype='<U16')

In [ ]:
# 比較對象：elephant.jpg
no=1
print(image_files[no])
similar_list = cosine_similarity(features2[no:no+1], features2[no+1:], dense_output=False)
print(np.sort(similar_list[0])[::-1])
image_files[no+1:][np.argsort(similar_list[0])[::-1]]

with-mask.jpg
[0.14363486 0.08666179 0.08474604 0.07914247 0.05226006 0.04416774
 0.04268286 0.04041187 0.03379241]


array(['without-mask.jpg', 'input.jpg', 'elephant2.jpg', 'style.jpg',
       '太陽花.jpg', 'bird01.jpg', 'Tiger2.jpg', 'elephant.jpg', 'Tiger.jpg'],
      dtype='<U16')